In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.vgg16 import preprocess_input
import tensorflow as tf

In [ ]:
img_width = 224
img_height = 224
batch_size = 128
num_of_classes = 70
IMG_SIZE = 224

training_set_directory = "/content/drive/MyDrive/Datasets/polishedGemstones/train"
validation_set_directory = "/content/drive/MyDrive/Datasets/polishedGemstones/val"
test_set_directory = "/content/drive/MyDrive/Datasets/polishedGemstones/test"

train_data = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)

validation_data = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

test_data = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

training_gen = train_data.flow_from_directory(
    training_set_directory,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    target_size=(img_height, img_width)
)

validation_gen = validation_data.flow_from_directory(
    validation_set_directory,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    target_size=(img_height, img_width)
)

test_gen = test_data.flow_from_directory(
    test_set_directory,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    target_size=(img_height, img_width)
)

Found 6542 images belonging to 70 classes.
Found 1857 images belonging to 70 classes.
Found 1000 images belonging to 70 classes.


In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


for layer in base_model.layers:
    layer.trainable = False

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_of_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:

epochs = 100

hist = model.fit(
    training_gen,
    epochs=epochs,
    validation_data=validation_gen,
    verbose=1,
    callbacks=[model_checkpoint_callback]
)

Epoch 1/10
52/52 [==============================] - 1658s 32s/step - loss: 8.1785 - accuracy: 0.0297 - val_loss: 3.9834 - val_accuracy: 0.0878
Epoch 2/10
52/52 [==============================] - 120s 2s/step - loss: 4.0620 - accuracy: 0.0556 - val_loss: 3.7725 - val_accuracy: 0.1002
Epoch 3/10
52/52 [==============================] - 118s 2s/step - loss: 3.9309 - accuracy: 0.0738 - val_loss: 3.6144 - val_accuracy: 0.1206
Epoch 4/10
52/52 [==============================] - 116s 2s/step - loss: 3.8375 - accuracy: 0.0786 - val_loss: 3.4571 - val_accuracy: 0.1325
Epoch 5/10
52/52 [==============================] - 127s 2s/step - loss: 3.7535 - accuracy: 0.0815 - val_loss: 3.3841 - val_accuracy: 0.1206
Epoch 6/10
52/52 [==============================] - 120s 2s/step - loss: 3.7084 - accuracy: 0.0809 - val_loss: 3.4473 - val_accuracy: 0.1131
Epoch 7/10
52/52 [==============================] - 119s 2s/step - loss: 3.6845 - accuracy: 0.0824 - val_loss: 3.3039 - val_accuracy: 0.1228
Epoch 8/10
